# Data Encoder
Uses an autoencoder to reduce the dimensionality of the data.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 11:14:59.463675: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 11:15:00.084914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 11:15:00.917456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:00.954670: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:00.954725: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The data should already be present as `dataset.csv` and `top_unigrams.txt` in the `data` folder.

If they are not present, do the following.
1. Ensure that the VirusTotal reports are present in `data/json` with the format `[LABEL]_[HASH].json`.
3. Run `prepare_data.py`. This will generate the two files needed for this notebook.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/dataset.csv")
raw_df

,label,hash,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ECHELON,5b3f895f7fc0f82147f36989f7c007dc73ea30297d2a53...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,UNKNOWN,6acde9d1d6882f75cdd14eeeb4821711734bd5cbee49c0...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HIVE,122e397dc3a55143bd276d6ff3bc04a05601fbf390aa52...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BABAR,7433bab5f01f1a93a2e40017bbbb209309a5fc14f12c80...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,STEAM,60f2ab502f2809fa7e33e43905c5b9e77080d4cf5d1f28...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


For the training of the model, we don't need the label or the file hash.

In [5]:
df = raw_df.drop(columns=["label", "hash"])
df

,dim-0000,dim-0001,dim-0002,dim-0003,dim-0004,dim-0005,dim-0006,dim-0007,dim-0008,dim-0009,...,dim-9990,dim-9991,dim-9992,dim-9993,dim-9994,dim-9995,dim-9996,dim-9997,dim-9998,dim-9999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


80% of the dataframe will be saved for training, while 20% will be left for testing.

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(
    df, test_size=0.2, random_state=42
)

In [7]:
X_train.shape

(800, 10000)

In [8]:
X_test.shape

(200, 10000)

# Model Creation

We will use an autoencoder to reduce the dimensionality of the data.

In [9]:
from keras import layers
from keras.models import Sequential

In [10]:
LAYER_SIZES = [2048, 512, 128, 32]  # The last layer is the center layer

In [11]:
INITIAL_EPOCH = 0

def create_encoder():
    model = Sequential(name="Encoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))

    for layer_size in LAYER_SIZES[:-1]:
        model.add(layers.Dense(layer_size, activation="relu"))

    # Add an activity regularizer to make the middle layer sparse
    model.add(layers.Dense(LAYER_SIZES[-1], activation="relu"))
    
    return model


def create_decoder():
    model = Sequential(name="Decoder")
    model.add(keras.Input((LAYER_SIZES[-1],), name="decoder-input"))

    for layer_size in LAYER_SIZES[-2::-1]:  # Starting from second last
        model.add(layers.Dense(layer_size, activation="relu"))
    model.add(layers.Dense(df.shape[1], activation="relu"))
    
    return model


def create_autoencoder(encoder, decoder):
    model = Sequential(name="Autoencoder")
    model.add(keras.Input((df.shape[1],), name="encoder-input"))
    model.add(encoder)
    model.add(decoder)

    model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"]
    )

    return model

In [12]:
encoder = create_encoder()
decoder = create_decoder()
autoencoder = create_autoencoder(encoder, decoder)

2024-04-29 11:15:02.176604: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:02.176735: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:02.176769: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:02.982442: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 11:15:02.982519: I external/local_xla/xla/stream_executor

In [13]:
autoencoder.summary(expand_nested=True)

Model: "Autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 32)             │    21,600,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 10000)          │    21,610,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_4 (Dense)            │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_5 (Dense)            │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_6 (Dense)            │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_7 (Dense)            │ (None, 10000)          │    20,490,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,211,824 (164.84 MB)

 Trainable params: 43,211,824 (164.84 MB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [14]:
import os

checkpoint_path = "../models/encoder/checkpoints/{epoch:04d}.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [15]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Define some utility functions that help identify the best model trained so far.

In [16]:
def find_latest_model(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)
    dir_contents = os.listdir(checkpoint_dir)
    if len(dir_contents) == 0:
        return None
    latest_model_path = sorted(dir_contents)[-1]
    return latest_model_path


def load_latest_model(checkpoint_dir):
    latest_model_path = find_latest_model(checkpoint_dir)
    if latest_model_path is None:
        print("No checkpoints found, not loading anything")
        return None
    print(f"Loading '{latest_model_path}'")
    model = keras.models.load_model(os.path.join(checkpoint_dir, latest_model_path))
    print("Done")
    return model

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 17
# autoencoder = load_latest_model(checkpoint_dir)

Train the model.

In [18]:
NUM_EPOCHS = 200

autoencoder.fit(
    X_train,
    X_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/200


I0000 00:00:1714360504.957948  835033 service.cc:145] XLA service 0x7fb0ec013880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714360504.958024  835033 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 11:15:04.991383: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 11:15:05.828556: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 5/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0202 - mae: 0.0275

I0000 00:00:1714360508.595718  835033 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0184 - mae: 0.0273
Epoch 1: val_loss improved from inf to 0.01594, saving model to ../models/encoder/checkpoints/0001.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 87ms/step - loss: 0.0183 - mae: 0.0271 - val_loss: 0.0159 - val_mae: 0.0231
Epoch 2/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0140 - mae: 0.0205
Epoch 2: val_loss improved from 0.01594 to 0.01380, saving model to ../models/encoder/checkpoints/0002.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0140 - mae: 0.0205 - val_loss: 0.0138 - val_mae: 0.0204
Epoch 3/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0124 - mae: 0.0188
Epoch 3: val_loss improved from 0.01380 to 0.01271, saving model to ../models/encoder/checkpoints/0003.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0123 - mae: 0.0187 - val_loss: 0.0127 - val_mae: 0.0187
Epoch 4/200
19/20 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0113 - mae: 0.0168
Epoch 4: val_loss improved from 0.01271 to 0.01

Load the best performing model.

In [19]:
autoencoder = load_latest_model(checkpoint_dir)

Loading '0042.keras'
Done


Evaluate the autoencoder on the test data.

In [20]:
test_mse, test_mae = autoencoder.evaluate(X_test, X_test, verbose=1)
print(f"Testing MSE: {test_mse:5.5f}")
print(f"Testing MAE: {test_mae:5.5f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - loss: 0.0074 - mae: 0.0098
Testing MSE: 0.00760
Testing MAE: 0.01000


Get only the encoder part to save.

In [21]:
encoder = autoencoder.get_layer("Encoder")

In [22]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2048)           │    20,482,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,600,928 (82.40 MB)

 Trainable params: 21,600,928 (82.40 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
encoder.save("../models/encoder/encoder.keras")

# Transforming Original Data

In [24]:
transformed_df = encoder.predict(df)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [25]:
transformed_df = pd.DataFrame(transformed_df, columns=[f"dim-{i:02d}" for i in range(LAYER_SIZES[-1])])

Add the labels and hashes back to the dataframe.

In [26]:
transformed_df.insert(0, "label", raw_df["label"])
transformed_df.insert(1, "hash", raw_df["hash"])

In [27]:
transformed_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,INSTALLBRAIN,a5ba68828e571de66675befdf4fbaf26dd226e25c2c703...,4.856040,9.959021,0.000000,5.447541,15.384576,0.000000,4.584654,0.000000,...,0.0,0.0,5.597068,18.320045,14.764016,0.000000,7.787311,14.470059,6.384356,0.000000
1,ECHELON,5b3f895f7fc0f82147f36989f7c007dc73ea30297d2a53...,1.294422,1.419385,0.000000,12.093320,0.228263,0.000000,4.231664,0.000000,...,0.0,0.0,0.000000,6.180888,12.521391,3.648764,0.000000,4.218205,0.000000,0.000000
2,UNKNOWN,6acde9d1d6882f75cdd14eeeb4821711734bd5cbee49c0...,0.000000,0.160647,0.000000,4.652483,0.000000,0.000000,0.000000,1.523965,...,0.0,0.0,0.263224,1.655028,5.390357,0.000000,0.000000,0.000000,0.000000,1.058898
3,HIVE,122e397dc3a55143bd276d6ff3bc04a05601fbf390aa52...,0.000000,0.000000,2.312020,5.555528,0.000000,0.000000,3.241326,1.654186,...,0.0,0.0,0.000000,0.000000,4.521163,3.160460,1.162947,0.000000,0.000000,0.000000
4,BABAR,7433bab5f01f1a93a2e40017bbbb209309a5fc14f12c80...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.491501,0.805943,...,0.0,0.0,6.028772,8.230105,5.101253,8.374162,21.305746,15.721295,19.534620,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,STEAM,60f2ab502f2809fa7e33e43905c5b9e77080d4cf5d1f28...,1.527257,2.814117,0.000000,14.827382,1.454555,0.000000,3.268779,0.000000,...,0.0,0.0,0.005216,8.933444,15.012127,2.206023,0.000000,3.774734,0.000000,0.000000
996,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,0.000000,0.000000,0.000000,4.795664,0.000000,0.000000,10.074182,0.344437,...,0.0,0.0,2.211108,0.350439,4.660242,11.047058,10.957180,3.310024,4.528979,0.000000
997,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0.000000,11.311330,0.000000,18.001150,0.945593,0.000000,6.007902,0.000000,...,0.0,0.0,1.039609,0.000000,2.340590,9.846880,0.000000,9.200933,0.000000,0.000000
998,UNKNOWN,08b4e7389242e3b8c37215a3b972f69193a9a12d5130bf...,0.000000,0.000000,0.000000,4.297989,0.000000,0.000000,0.000000,1.048411,...,0.0,0.0,0.000000,1.956394,4.991206,0.331093,0.000000,1.939405,0.000000,0.000000


In [28]:
transformed_df.to_csv("../data/encoded-data.csv", index=False)